# unlabel_L2_MLP_predict
  - input
      -  Syndigo_Levels_L1_L2_L3.zip
      -  Inactive GTINs as of Sept 6.xlsx

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    # print(f'key: {key}') 
    # print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1_label():
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL

    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 

    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped

In [4]:
def prepare_data_1_unlabel():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time() 
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "Syndigo_Levels_L1_L2_L3.zip",compression="zip") # (5211106, 22)    
    # Remove inactive GTIN 
    print(f'Before drop inactive GTIN, pim_gtin_mapped.shape {pim_gtin_mapped.shape}')    
    df = pd.read_excel(DBFR + "Inactive GTINs as of Sept 6.xlsx")
    l_gtin_inc = list(df['GTIN|Ecommerce Description'].apply(lambda x:  x.split('|')[0]))
    l_gtin_ex = [float(x) for x in l_gtin_inc]
    pim_gtin_mapped = pim_gtin_mapped[~pim_gtin_mapped.GTIN.isin(l_gtin_ex)]     
    print(f'After drop inactive GTIN, pim_gtin_mapped.shape {pim_gtin_mapped.shape}')    

    
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
         pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    pim_gtin_mapped.drop_duplicates('GTIN', inplace = True)
    pim_gtin_mapped['ITEM_SUBCOM_text'] = \
    (pim_gtin_mapped.VND_ECOM_DSC + ' ' + pim_gtin_mapped.SUBCOM_DSC).fillna('').str.lower()
  
    return pim_gtin_mapped

In [22]:
def prepare_data_2(level_1):

    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
    series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  == 1 ].index.tolist())].index)  
    
    x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    subset_df_unl = pim_gtin_mapped[pim_gtin_mapped['Level1'] == level_1]
    subset_df_unl.reset_index(inplace = True, drop=True)
    X  = np.array(list(subset_df_unl.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' '))))) 
    
    return  x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map, subset_df, X, subset_df_unl

In [6]:
def prepare_data_2_unlabel(level_1):

    subset_df_unl = pim_gtin_mapped[pim_gtin_mapped['Level 1'] == level_1]
    
    X  = np.array(list(subset_df_unl.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  x_subset_w2v, y

In [7]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [8]:

def proc_pred_mlp(level_1):
     global df_label, df_unlabel, mlp
     mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
     #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
     print(f"Training - w2v + MLP \"{level_1}\"")
     warnings.filterwarnings("ignore", category=UserWarning)
     mlp.fit(X_train, y_train)
     print(' Display MLP test  metrics')
     preds_test_mlp = mlp.predict(X_test)
     #df_test = proc_prob(X_test, A_test, desc= "Test")   
                         
     preds_train_mlp = mlp.predict(X_train)
     #df_train = proc_prob(X_test, A_test, desc= "Train")     
                          
     df_unl  = proc_prob(X, df_subset_unl)
     #df_label  = df_label +  df_test +  df_train    
    
     df_unlabel  = pd.concat([df_unlabel, df_unl])
     # print(classdf_label +nification_report(y_test, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))      
     return 


In [9]:
def proc_prob(mdl_data, df_base, desc= False):

    preds = list(mlp.predict(mdl_data)) 
    probx = mlp.predict_proba(mdl_data)
    probs = []
    for i in range(len(mdl_data)):
      probs.append(probx[i][np.argsort(probx[i])][::-1][:1])
    probs = [element for sublist in list(probs) for element in sublist]
    data = {'Predict': preds, 'Scores': probs}
    df_pred= pd.DataFrame(data)
    if desc == False: pass
    else:
       df_pred['Type'] = desc
    df_pred['Predict'] = df_pred['Predict'].fillna('Other').map(id2_level_map) 
    df_merg = pd.merge( df_base, df_pred, left_index=True, right_index=True)
    return df_merg

In [18]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from datetime import date
from sklearn.neural_network import MLPClassifier


from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect, pickle
import warnings
import logging
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.4f}'.format 
pd.set_option("display.max_rows", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [11]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path

In [12]:

syndigo_mapped= prepare_data_1_label()
pim_gtin_mapped = prepare_data_1_unlabel()

syndigo_mapped.shape (259085, 24)
syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & 

In [23]:
start_time = time.time() 

### import word2vec model trained on entire PIMMART data
#model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
model.most_similar('chips')
adj_ratio = 0.2
i =0
err_smote_cnt = 0
err_smp_cnt = 0
#N = 5 
dic_level1= constrcut_dic()
#dic_level1 = dict(list(dic_level1.items())[3: 5])
dic_scure={}
l_metrics = []
l_metrics_v = []
l_no_mdl = []
df_unlabel = pd.DataFrame() 
df_label = pd.DataFrame() 


for level_1, _ in dic_level1.items():    
    print(f'{i}. Process level 1 = {level_1}')    
    i += 1    
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map, df_subset, X, df_subset_unl =  prepare_data_2(level_1)
    
    if  len ( x_subset_w2v) < 10: 
        len_x_subset_w2v = len(x_subset_w2v)
        print(f'len(x_subset_w2v) = {len_x_subset_w2v}. Not  process len(x_subset_w2v) < 10')
        l_no_mdl.append([level_1, 'Not  process len of data  < 10']) 
        continue
    dic_y = dict(pd.Series(y_subset_w2v).value_counts())  
    if len(dic_y.items())  == 1:
       print(f'Level1 {level_1} only has one level2. Will skip SMOTE' )
       l_no_mdl.append([level_1, 'Only has one level2'])  
       continue
    try:
       #A_train, A_test, B_train, B_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42) 
       X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)
  
       proc_pred_mlp(level_1)
    except Exception as smp_ve:
       err_smp_cnt += 1 
       len_x_subset_w2v  = len ( x_subset_w2v)
       logging.exception("number of smaple error %d. len ( x_subset_w2v) = %d.  Exception occurred: %s", err_smp_cnt , len_x_subset_w2v,  str(smp_ve))
        
end_time = time.time()   
desc = f'Elapse_time for evluating LR, MLP and MLP_SMOTE'
elapse_time (  start_time, end_time, desc)

0. Process level 1 = Food / Beverages
Training - w2v + MLP "Food / Beverages"
 Display MLP test  metrics
1. Process level 1 = Health & Beauty
Training - w2v + MLP "Health & Beauty"
 Display MLP test  metrics
2. Process level 1 = Beer / Wine / Spirits
Training - w2v + MLP "Beer / Wine / Spirits"
 Display MLP test  metrics
3. Process level 1 = Kitchen & Bathroom
Training - w2v + MLP "Kitchen & Bathroom"
 Display MLP test  metrics
4. Process level 1 = Cleaning & Janitorial
Training - w2v + MLP "Cleaning & Janitorial"
 Display MLP test  metrics
5. Process level 1 = Home & Venue Decoration
Training - w2v + MLP "Home & Venue Decoration"
 Display MLP test  metrics
6. Process level 1 = Livestock & Pet Supplies
Training - w2v + MLP "Livestock & Pet Supplies"
 Display MLP test  metrics
7. Process level 1 = Toys / Games / Hobbies
Training - w2v + MLP "Toys / Games / Hobbies"
 Display MLP test  metrics
8. Process level 1 = Childcare
Training - w2v + MLP "Childcare"
 Display MLP test  metrics
9. Pr

ERROR:root:number of smaple error 1. len ( x_subset_w2v) = 12.  Exception occurred: The test_size = 1 should be greater or equal to the number of classes = 2
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_13040\3530044818.py", line 41, in <module>
    proc_pred_mlp(level_1)
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_13040\2984912218.py", line 7, in proc_pred_mlp
    mlp.fit(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 747, in fit
    return self._fit(X, y, incremental=False)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 471, in _fit
    self._fit_stochastic(
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 583, in _fit_stochastic
    X, X_val, y, y_val = train_test_split(
  File "C:\Users\in

Training - w2v + MLP "Marine"
31. Process level 1 = Flooring
len(x_subset_w2v) = 9. Not  process len(x_subset_w2v) < 10
32. Process level 1 = Paints & Coatings
Training - w2v + MLP "Paints & Coatings"
 Display MLP test  metrics
33. Process level 1 = Building Supplies
Training - w2v + MLP "Building Supplies"
 Display MLP test  metrics
34. Process level 1 = Musical Instruments
len(x_subset_w2v) = 0. Not  process len(x_subset_w2v) < 10
35. Process level 1 = Material Handling
len(x_subset_w2v) = 3. Not  process len(x_subset_w2v) < 10
36. Process level 1 = Agricultural Equipment
len(x_subset_w2v) = 2. Not  process len(x_subset_w2v) < 10
 Elapse_time for evluating LR, MLP and MLP_SMOTE It took 218.328427 seconds - 0hh:3mm:38ss.
 start time: Sep 13 2023 22:27:50  end time:  Sep 13 2023 22:31:29


(0, 3, 38)

In [28]:
pim_gtin_mapped = pd.read_csv(DBFR + "Syndigo_Levels_L1_L2_L3.zip",compression="zip")
zip_file = path + "mlp_unlbel_scores_"+dte + '.zip'
df_unlabel.to_csv(zip_file,compression="zip")

In [20]:
df_unlabel.head()

,VND_ECOM_DSC,Level1,L1_Scores,Level2,L2_Scores,GTIN,PMY_DPT_CD,PMY_DPT_DSC,REC_DPT_CD,REC_DPT_DSC,...,SUBCOM_DSC,ITM_ID,L1_Percent,L2_Percent,L2_Flag,Score,Level3,ITEM_SUBCOM_text,Predict,Scores
0,Shade Mate 7 Ft. Umbrella w/ Instant Table S20...,Cleaning & Janitorial,0.3233,Cleaning Chemicals,0.3479,"90,026,619,252,213.0000",3.0000,DRUG/GM,87.0000,GM,...,PATIO SEATING,"745,140,465.0000",<50,<50,1,0.8000,Acrylic Cleaners,shade mate 7 ft. umbrella w/ instant table s20...,Cleaning Chemicals,0.9126
1,FTI Brands ERASERS Product Line Floor Display ...,Cleaning & Janitorial,0.7508,Cleaning Chemicals,0.3716,"80,867,745,000,209.0000",3.0000,DRUG/GM,87.0000,GM,...,SHOE CARE & LACES,"746,670,044.0000",70-80,<50,1,0.8000,Acrylic Cleaners,fti brands erasers product line floor display ...,Cleaning Chemicals,0.6351
2,Pop & Load Collapsible Laundry Basket,Cleaning & Janitorial,0.9979,Cleaning Chemicals,0.5400,"80,812,229,030,333.0000",3.0000,DRUG/GM,87.0000,GM,...,LAUNDRY,"743,708,132.0000",90-100,50-60,1,0.8000,Acrylic Cleaners,pop & load collapsible laundry basket laundry,Storage & Organization,0.9989
3,TROPICAL 6 in ASST in WRAP 8PK,Cleaning & Janitorial,0.2530,Cleaning Chemicals,0.2492,"80,727,809,615,254.0000",7.0000,PRODUCE,18.0000,FLORAL,...,HOUSEPLANTS 6IN & LARGER (2QT & LARGER),"746,303,296.0000",<50,<50,1,0.8000,Acrylic Cleaners,tropical 6 in asst in wrap 8pk houseplants 6in...,Air Fresheners,0.9822
4,LILY 6 in CALLA BRIGHT TIN 8PK,Cleaning & Janitorial,0.2282,Cleaning Chemicals,0.3243,"80,727,809,531,288.0000",7.0000,PRODUCE,18.0000,FLORAL,...,BULBS INDOOR,"745,100,912.0000",<50,<50,1,0.8000,Acrylic Cleaners,lily 6 in calla bright tin 8pk bulbs indoor,Air Fresheners,0.9793


In [1]:
syndigo_mapped['Level 1'].value_counts(dropna = False) 

NameError: name 'syndigo_mapped' is not defined